# File Interaction and Q&A:
This agent is designed to interact with a local file system to perform three primary tasks:

1. List files in a specified directory
2. Read file contents
3. Answer user questions based on the content of those files

# The agent loop 
The Agent Loop is the core engine driving an AI agent’s decision-making process.

It allows the agent to interact dynamically with its environment by continuously generating actions, executing them, and adapting based on the results.

This loop makes the agent capable of completing complex tasks step by step.

In [1]:
import os
from dotenv import load_dotenv

# Get API key from environment variables
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

if api_key:
    print("API key loaded successfully!")
else:
    print("Error: API key not found in .env file")


API key loaded successfully!


In [2]:
from litellm import completion
from typing import List, Dict
import sys
import os
import json
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
      model="openai/gpt-4o",
      messages=messages,
      max_tokens=1000
   )
    return response.choices[0].message.content


In [3]:
def parse_action(response):
    """Parse the agent's response to extract the action and its parameters."""
    try:
        # Extract the action JSON from between ```action and ``` tags
        action_text = response.split("```action")[1].split("```")[0].strip()
        action = json.loads(action_text)
        return action
    except Exception as e:
        print(f"Error parsing action: {e}")
        return {
            "tool_name": "error",
            "args": {"message": "Failed to parse the action. Make sure the response follows the correct format."}
        }

In [4]:
def list_files():
    """List all files in the current directory."""
    try:
        files = [f for f in os.listdir('.') if os.path.isfile(f)]
        return files
    except Exception as e:
        return f"Error listing files: {e}"

In [5]:
def read_file(file_name):
    """Read the content of a file."""
    try:
        if not os.path.exists(file_name):
            return f"Error: File '{file_name}' does not exist."
        
        with open(file_name, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        return f"Error reading file: {e}"

In [6]:
def run_agent(task, max_iterations=10):
    """Run the agent loop to perform the specified task."""
    # Define agent rules
    agent_rules = [{
        "role": "system",
        "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:
- list_files() -> List[str]: List all files in the current directory.
- read_file(file_name: str) -> str: Read the content of a file.
- terminate(message: str): End the agent loop and print a summary to the user.

If a user asks about files, list them before reading.

Every response MUST have an action.
Respond in this format:

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```
"""
    }]
    
    # Initialize memory with the user's task
    memory = [
        {"role": "user", "content": task}
    ]
    
    iterations = 0
    
    # The Agent Loop
    while iterations < max_iterations:
        # 1. Construct prompt: Combine agent rules with memory
        prompt = agent_rules + memory
        
        # 2. Generate response from LLM
        print("\nAgent thinking...")
        response = generate_response(prompt)
        print(f"Agent response: {response}")
        
        # 3. Parse response to determine action
        action = parse_action(response)
        
        # 4. Execute action
        if action["tool_name"] == "list_files":
            result = {"result": list_files()}
        elif action["tool_name"] == "read_file":
            result = {"result": read_file(action["args"]["file_name"])}
        elif action["tool_name"] == "terminate":
            print(f"\nAgent terminated: {action['args']['message']}")
            break
        elif action["tool_name"] == "error":
            result = {"error": action["args"]["message"]}
        else:
            result = {"error": f"Unknown action: {action['tool_name']}"}
        
        print(f"Action result: {result}")
        
        # 5. Update memory with response and results
        memory.extend([
            {"role": "assistant", "content": response},
            {"role": "user", "content": json.dumps(result)}
        ])
        
        # 6. Check termination condition
        if action["tool_name"] == "terminate":
            break
        
        iterations += 1
    
    if iterations >= max_iterations:
        print(f"\nReached maximum iterations ({max_iterations}). Agent loop terminated.")


In [10]:
if __name__ == "__main__":
    # Example usage
    user_task = input("What would you like the agent to do? ")
    run_agent(user_task)


Agent thinking...
Agent response: ```action
{
    "tool_name": "list_files",
    "args": {}
}
```
Action result: {'result': ['GivingAgentsMemory.ipynb', 'CustomerServiceAgent.ipynb', 'refined_email.txt', 'Building_Quasi_Agent.ipynb', 'initial_draft.txt', 'variant_email.txt', 'AI_Agent.ipynb', 'function_that_calculates_the_f.py']}

Agent thinking...
Agent response: ```action
{
    "tool_name": "terminate",
    "args": {
        "message": "The file 'to_do.txt' does not exist in the current directory."
    }
}
```

Agent terminated: The file 'to_do.txt' does not exist in the current directory.


In [11]:
import os
import json
import litellm

# Set your API key for the LLM service
# Uncomment and set your API key:
# os.environ["OPENAI_API_KEY"] = "your-api-key"

def generate_response(messages):
    """Generate a response from the LLM based on the provided messages."""
    try:
        response = litellm.completion(
            model="gpt-3.5-turbo",  # You can change to a different model
            messages=messages,
            temperature=0.2
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error: Unable to generate a response."

def parse_action(response):
    """Parse the agent's response to extract the action and its parameters."""
    try:
        # Try to find action format with ```action tags
        if "```action" in response:
            action_text = response.split("```action")[1].split("```")[0].strip()
            action = json.loads(action_text)
            return action
        
        # Alternative format: just the word "action" followed by JSON
        elif "action" in response:
            # Find the start of JSON after "action"
            start_idx = response.find("{", response.find("action"))
            if start_idx != -1:
                # Find the end of the JSON block (matching closing brace)
                open_braces = 0
                for i in range(start_idx, len(response)):
                    if response[i] == "{":
                        open_braces += 1
                    elif response[i] == "}":
                        open_braces -= 1
                        if open_braces == 0:
                            end_idx = i + 1
                            break
                
                action_text = response[start_idx:end_idx]
                action = json.loads(action_text)
                return action
        
        # If neither format is found, raise an exception
        raise ValueError("Could not find action format in response")
        
    except Exception as e:
        print(f"Error parsing action: {e}")
        return {
            "tool_name": "error",
            "args": {"message": "Failed to parse the action. Make sure the response follows the correct format."}
        }

def list_files():
    """List all files in the current directory."""
    try:
        files = [f for f in os.listdir('.') if os.path.isfile(f)]
        return files
    except Exception as e:
        return f"Error listing files: {e}"

def read_file(file_name):
    """Read the content of a file."""
    try:
        if not os.path.exists(file_name):
            return f"Error: File '{file_name}' does not exist."
        
        with open(file_name, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        return f"Error reading file: {e}"

def run_agent(task, max_iterations=10):
    """Run the agent loop to perform the specified task."""
    # Define agent rules
    agent_rules = [{
        "role": "system",
        "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:
- list_files() -> List[str]: List all files in the current directory.
- read_file(file_name: str) -> str: Read the content of a file.
- terminate(message: str): End the agent loop and print a summary to the user.

If a user asks about files, list them before reading.

Every response MUST have an action.
Respond in this format:

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```
"""
    }]
    
    # Initialize memory with the user's task
    memory = [
        {"role": "user", "content": task}
    ]
    
    iterations = 0
    
    # The Agent Loop
    while iterations < max_iterations:
        # 1. Construct prompt: Combine agent rules with memory
        prompt = agent_rules + memory
        
        # 2. Generate response from LLM
        print("\nAgent thinking...")
        response = generate_response(prompt)
        print(f"Agent response: {response}")
        
        # 3. Parse response to determine action
        action = parse_action(response)
        
        # 4. Execute action
        if action["tool_name"] == "list_files":
            result = {"result": list_files()}
        elif action["tool_name"] == "read_file":
            result = {"result": read_file(action["args"]["file_name"])}
        elif action["tool_name"] == "terminate":
            print(f"\nAgent terminated: {action['args']['message']}")
            break
        elif action["tool_name"] == "error":
            result = {"error": action["args"]["message"]}
        else:
            result = {"error": f"Unknown action: {action['tool_name']}"}
        
        print(f"Action result: {result}")
        
        # 5. Update memory with response and results
        memory.extend([
            {"role": "assistant", "content": response},
            {"role": "user", "content": json.dumps(result)}
        ])
        
        # 6. Check termination condition
        if action["tool_name"] == "terminate":
            break
        
        iterations += 1
    
    if iterations >= max_iterations:
        print(f"\nReached maximum iterations ({max_iterations}). Agent loop terminated.")

if __name__ == "__main__":
    # Interactive session
    print("AI Agent is ready! Type 'exit' to quit.")
    
    while True:
        user_task = input("\nWhat would you like the agent to do? ")
        
        if user_task.lower() in ['exit', 'quit', 'bye']:
            print("Exiting agent. Goodbye!")
            break
            
        run_agent(user_task)

AI Agent is ready! Type 'exit' to quit.

Agent thinking...
Agent response: action
{
    "tool_name": "list_files"
}
Error parsing action: list index out of range
Action result: {'error': 'Failed to parse the action. Make sure the response follows the correct format.'}

Agent thinking...
Agent response: action
{
    "tool_name": "list_files"
}
Error parsing action: list index out of range
Action result: {'error': 'Failed to parse the action. Make sure the response follows the correct format.'}

Agent thinking...
Agent response: action
{
    "tool_name": "list_files"
}
Error parsing action: list index out of range
Action result: {'error': 'Failed to parse the action. Make sure the response follows the correct format.'}

Agent thinking...
Agent response: action
{
    "tool_name": "list_files"
}
Error parsing action: list index out of range
Action result: {'error': 'Failed to parse the action. Make sure the response follows the correct format.'}

Agent thinking...
Agent response: action
{
